# 第3周课后作业
## 一元变量的线性回归

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time # 用来记录算法的计算时间

plt.rcParams.update({
    "font.family": ["Times New Roman"],  # 使用 Times 字体
    "mathtext.fontset": "stix",        # 数学字体匹配 Times
    "font.size": 12                    # 默认字号
})

### 习题1
在对一个统计模型进行蒙特卡洛模拟时，我们通常会考虑多种不同的设置。为了简化讨论，我们仅考虑样本量变化的情况。设样本量$n$分别取$\{100, 300, 500, 1000\}$，并使用图表来汇报以下结果。请注意，在所有图表中，样本量应作为$x$轴。

#### 1.1 偏差盒状图
以$y$轴表示偏差，分别绘制并讨论两个参数估计量在$1000$次蒙特卡洛模拟中的偏差。

注：只需完善函数`plot_bias_boxplots_grouped`及相关调用。

In [ ]:
def train_data_generation(n, rn):
    """
    生成样本数据 (x, y)

    参数:
        n : 样本量
        rn: 随机种子

    返回:
        x : n*1 的特征向量，从 N(2, 2^2) 中生成
        y : n*1 的标签向量，满足 y = 1 + x + ε，其中 ε ~ N(0,1)
    """
    np.random.seed(rn)
    x = np.random.normal(2, 2, (n,1))
    epsilon = np.random.normal(0, 1, (n,1))
    y = 1 + x + epsilon

    return x, y #打包输出相关结果

def estimation_summation(x,y):
    """
    使用逐项求和方法计算 OLS 估计量

    参数:
        x : n*1 特征向量
        y : n*1 标签向量

    返回:
        par_est : 长度为2的一维数组,[截距项估计值, 斜率项估计值]
    """

    n = len(x)
    #将截距项1加入到特征中，得到增广特征矩阵，记为aug_x
    aug_x = np.concatenate((np.ones_like(x),x), axis = 1)
    xx = np.zeros((2,2))
    xy = np.zeros((2,1))

    #计算'xx'和'xy'的求和
    for i in range(n):
        example_i = aug_x[i,:].reshape((2,1))
        xx += example_i @ example_i.transpose()
        xy += example_i * y[i]
    #计算'xx'的逆矩阵
    xx_inv = np.linalg.inv(xx)
    #得到估计量
    par_est = xx_inv @ xy
    
    return par_est.flatten()

def estimation_vectorization(x,y):
    """
    使用矩阵向量化方法计算 OLS 估计量

    参数:
        x : n*1 特征向量
        y : n*1 标签向量

    返回:
        par_est : 长度为2的一维数组[截距项估计值, 斜率项估计值]
    """
    #将截距项1加入到特征中，得到增广特征矩阵，记为aug_x
    aug_x = np.concatenate((np.ones_like(x),x), axis = 1)
    #通过X^TX获得'xx'，其中X是步骤2得到的增广矩阵
    xx = aug_x.transpose() @ aug_x
    #通过X^TY获得'xy'，其中Y的形状为nX1
    xy = aug_x.transpose() @ y
    #计算'xx'的逆矩阵
    xx_inv = np.linalg.inv(xx)  
    #获得估计器
    par_est = xx_inv @ xy 

    return par_est.flatten()

def calculate_bias(sample_sizes=(100, 300, 500, 1000), replications=1000, method=None):
    """
    进行蒙特卡洛模拟并计算 OLS 估计量的偏差

    参数:
        sample_sizes : 元组，表示要模拟的样本量集合
        replications : 每个样本量下的重复次数
        method       : 用于估计参数的函数

    返回:
        bias_results ：字典｛样本量：偏差数组 维度(replications,2)}
    """
    # 参数真值
    true_beta = np.array([1.0, 1.0])
    # 初始化结果容器
    bias_results = {n: np.zeros((replications, 2)) for n in sample_sizes}
    # 遍历样本量
    for n in sample_sizes:
        # 在每个样本量下进行 replications 次重复模拟
        for r in range(replications):
            # 生成数据
            x, y = train_data_generation(n=n, rn=r)
            # 调用指定估计方法
            estimates = method(x, y)
            # 保存偏差
            bias_results[n][r,:] = estimates - true_beta

    return bias_results

In [ ]:
def plot_bias_boxplots_grouped(bias_results):
    """
    绘制偏差的分组盒图 (grouped boxplot)

    参数:
        bias_results : 字典｛样本量：偏差数组 维度 (replications,2)}
                      
    返回:
        None
    """
    # 提取样本量及对应的截距、斜率偏差数据
    ns = sorted(bias_results.keys())
    bias_intercept = [bias_results[n][:, 0] for n in ns]
    bias_slope     = # 请完善

    # 初始化画布
    fig, ax = plt.subplots(figsize=(6.5, 4.5))
    positions = np.arange(len(ns)) * 3  # 每组盒图的横坐标位置

    # 截距盒图（左边）
    bp1 = ax.boxplot(
        bias_intercept,
        positions=positions - 0.4,
        widths=0.8,
        patch_artist=True,
        boxprops=dict(facecolor="lightgray")
    )

    # 斜率盒图（右边）
    bp2 = ax.boxplot(





    ) # 请完善
    # 添加零基准线
    ax.axhline(0, linestyle="--", color="black", linewidth=1)
    # 设置坐标轴、标签、图例
    ax.set_xticks(positions)
    ax.set_xticklabels([str(n) for n in ns])
    ax.set_xlabel("Sample size")
    ax.set_ylabel("Bias")
    ax.legend([bp1["boxes"][0], bp2["boxes"][0]], ["Intercept", "Slope"], loc="upper right")
    # 调整布局并显示
    plt.tight_layout()
    plt.show()

In [ ]:
# 绘制求和方法的偏差盒状图
bias_summation = 
plot_bias_boxplots_grouped(bias_summation)

In [ ]:
# 绘制向量化方法的偏差盒状图
bias_vectorization = 
plot_bias_boxplots_grouped(bias_vectorization)

#### 1.2. 计算效率图表
展示两种估计方法的计算效率，不同方法应使用不同的颜色。绘制图表并对结果进行评论。

注：只需完善函数`plot_time_comparison`及相关调用。

In [ ]:
def calculate_time(sample_sizes=(100, 300, 500, 1000), replications=1000, method=None):
    """
    计算不同样本量下某种估计方法的运行时间

    参数:
        sample_sizes : 元组或列表，要测试的样本量
        replications : int, 每个样本量下的重复次数
        method : 估计方法函数

    返回:
        time_results : 字典 {样本量：耗时}
    """
    # 初始化结果容器
    time_results = {}
    # 遍历不同样本量
    for n in sample_sizes:
        # 记录开始时间
        T1 = time.time()
        for r in range(replications):
            # 生成数据，随机种子为r
            x, y = train_data_generation(n=n, rn=r)
            # 执行指定估计方法
            par_est = method(x,y)
        # 记录结束时间
        T2 = time.time()
        # 记录耗时
        time_results[n] = T2 - T1
        print(f'样本量:{n}, 方法:{method.__name__}, 耗时{time_results[n]:.4f}秒')
    return time_results

In [ ]:
def plot_time_comparison(times_sum, times_vec):
    """
    绘制两种方法的计算效率比较折线图

    参数:
        times_sum : 字典 {样本量: 耗时}
        times_vec : 字典 {样本量: 耗时}

    返回:
        None
    """
    ns = sorted(times_sum.keys())
    time_sum = [times_sum[n] for n in ns]
    time_vec = # 请完善
    
    # 初始化画布
    plt.figure(figsize=(6.5, 4.5))
     # 绘制 Summation 方法的折线图
    plt.plot(ns, time_sum, marker="o", color="black", linestyle="-", label="Summation")
    # 绘制 Vectorization 方法的折线图
    plt.plot() # 请完善

    # 设置坐标轴和标题
    plt.xlabel("Sample size (n)")
    plt.ylabel("Computation time (seconds)")
    plt.title("Computation Efficiency Comparison")
    plt.legend()
    
    # 调整布局并显示
    plt.tight_layout()
    plt.show()

In [ ]:
times_vectorization = 
times_summation = 
plot_time_comparison(times_summation, times_vectorization)

### 习题2

考虑 $n \in \{500, 1000, 1500, 2000, 3000\}$等5种不同的情况，计算基于求和法和向量法的所需要的计算时间，并将两种方法所需要的计算时间画在一张图中，并简要分析你所得到的结果。

注：只需调用`calculate_time`函数

In [ ]:
times_vectorization = 
times_summation = 
plot_time_comparison(times_summation, times_vectorization)

### 习题3
结合模型
$$ y_i = b_0 + w_0 x_i + \epsilon_i \quad (i = 1, \cdots, n), \tag{1}$$
其中 
$\epsilon_i \sim N(0, 1)$
为白噪声以及估计量
$$ (\hat{b},\hat{w})
=\left(\sum_{i=1}^{n}\boldsymbol{\tilde{x}_{i}}(\boldsymbol{\tilde{x}_{i}})^{\mathrm{T}}\right)^{-1}\sum_{i=1}^{n}\boldsymbol{\tilde{x}_{i}}y_{i}
=(\boldsymbol{X}^{\mathrm{T}}\boldsymbol{X})^{-1}\boldsymbol{X}^{\mathrm{T}}\boldsymbol{Y}. \tag{4} $$
其中 
$\boldsymbol{\tilde{x}_{i}} = (1, x_i)^{\mathrm{T}}$, $\boldsymbol{X}=(\tilde{\boldsymbol{x}}_i,\ldots,\tilde{\boldsymbol{x}}_n)^\mathrm{т}\in\mathbb{R}^{n\times2}$ 
以及 
$\boldsymbol{Y}=(y_1,\ldots,y_n)^\mathrm{T}$. 
证明 (4) 中的估计量是统计无偏的，即 $E(\hat b) = b_0$ 以及 $E(\hat w) = w_0$.

解：请输入$\LaTeX$或者在此处粘贴手写图片

# 回归模型

### 习题1
基于估计量
$$\hat{\boldsymbol{\theta}}=(\boldsymbol{X}^{\mathrm{T}}\boldsymbol{X})^{-1}\boldsymbol{X}^{\mathrm{T}}\boldsymbol{Y} \tag{11}$$
所得到的 $\hat{\boldsymbol{\theta}}$ 是否满足 $\sum_{i=1}^n(y_i-\tilde{\boldsymbol{x}}_i^\mathrm{T}\hat{\boldsymbol{\theta}})=0$？为什么？

解：请输入$\LaTeX$或者在此处粘贴手写图片

### 习题2

记 
$\boldsymbol{x}=(x_1,\ldots,x_d)^\mathrm{т}\in\mathbb{R}^d$, 
$\boldsymbol{a}=(a_1,\ldots,a_d)^\mathrm{т}\in\mathbb{R}^d$, 
$\boldsymbol{A}=(a_{ij})\in\mathbb{R}^{d\times d}$ 回答如下问题：

### 2.1 
假设向量 $\boldsymbol{a}$ 为定值，考虑关于 $\boldsymbol{x}$ 的函数 $f(\boldsymbol{x})=\boldsymbol{x}^\mathrm{T}\boldsymbol{a}=\sum_{i=1}^dx_ia_i$。对于$i=1,\ldots,d,$ 计算 $\partial f(\boldsymbol{x})/\partial x_i$ 并证明如下结论：
$$\frac{\partial f(\boldsymbol{x})}{\partial \boldsymbol{x}} = \boldsymbol{a}.$$

解：请输入$\LaTeX$或者在此处粘贴手写图片

#### 2.2 
假设矩阵 $\boldsymbol{A}$为定值，考虑关于 $\boldsymbol{x}$的函数
$g(\boldsymbol{x})=\boldsymbol{x}^\mathrm{T}\boldsymbol{A}\boldsymbol{x}=\sum_{i=1}^d\sum_{j=1}^da_{ij}x_ix_j$。
对于$i=1,\ldots,d$ 以及 $j=1,\ldots,d$ 计算 
$\partial g(\boldsymbol{x})/\partial x_i$ 
以及 
$\partial^2g(\boldsymbol{x})/(\partial x_i\partial x_j)$，
并证明如下两个结论：
$$ \frac{\partial g(\boldsymbol{x})}{\partial\boldsymbol{x}}=(\boldsymbol{A}+\boldsymbol{A}^\mathrm{T})\boldsymbol{x}, $$
$$ \frac{\partial^2g(\boldsymbol{x})}{\partial\boldsymbol{x}^\mathrm{T}\partial\boldsymbol{x}}=\boldsymbol{A}+\boldsymbol{A}^\mathrm{T}。 $$

解：请输入$\LaTeX$或者在此处粘贴手写图片

#### 2.3

结合代价函数
$$
\mathcal{J}(\boldsymbol{\theta})=\frac{1}{n}\sum_{i=1}^{n}(y_{i}-\tilde{\boldsymbol{x}}_{i}^{\mathrm{T}}\boldsymbol{\theta})^{2}=\frac{1}{n}(\boldsymbol{Y}-\boldsymbol{X}\boldsymbol{\theta})^{\mathrm{T}}(\boldsymbol{Y}-\boldsymbol{X}\boldsymbol{\theta}), \tag{8}
$$
以及一阶条件
$$
\frac{\partial\mathcal{J}}{\partial\boldsymbol{\theta}}(\hat{\boldsymbol{\theta}})=0 \tag{9}
$$
证明OLS参数估计量$\boldsymbol{\hat \theta}$为
$$
\hat{\boldsymbol{\theta}}=(\boldsymbol{X}^{\mathrm{T}}\boldsymbol{X})^{-1}\boldsymbol{X}^{\mathrm{T}}\boldsymbol{Y}
$$

解：请输入$\LaTeX$或者在此处粘贴手写图片